In [1]:
import pandas as pd
import numpy as np
import random
import os
import torch
import sys
sys.path.append('../')
from modules import utils, constants
import warnings
warnings.filterwarnings('ignore')

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

#### The data

In [3]:
# train_df = pd.read_csv('../data/25_jan/train_set_basic.csv')
train_df = pd.read_csv('../data/missingness/0/training_set.csv')
train_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,1,0,0,0,0,0,0,1,1,0,...,0,1,0,0,0,0,0,1,1,1
4,1,0,0,1,1,1,0,0,0,0,...,0,1,0,0,0,1,0,0,1,1


In [4]:
train_df.cutaneous_lupus.value_counts()

0    44200
3     3120
1     2246
2      834
Name: cutaneous_lupus, dtype: int64

In [5]:
train_df[(train_df.ana==0) & (train_df.label==1)]

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label


In [6]:
train_df.isna().sum()

ana                             0
fever                           0
leukopenia                      0
thrombocytopenia                0
auto_immune_hemolysis           0
delirium                        0
psychosis                       0
seizure                         0
non_scarring_alopecia           0
oral_ulcers                     0
cutaneous_lupus                 0
pleural_effusion                0
pericardial_effusion            0
acute_pericarditis              0
joint_involvement               0
proteinuria                     0
anti_cardioliphin_antibodies    0
anti_β2gp1_antibodies           0
lupus_anti_coagulant            0
low_c3                          0
low_c4                          0
anti_dsdna_antibody             0
anti_smith_antibody             0
label                           0
dtype: int64

In [7]:
len(train_df)

50400

In [8]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_train, y_train = np.array(X_train), np.array(y_train)
X_train.shape, y_train.shape

((50400, 23), (50400,))

In [9]:
action_list = list(constants.CLASS_DICT.keys()) + [col  for col in train_df.columns if col!='label']
action_list

['No lupus',
 'Lupus',
 'Inconclusive diagnosis',
 'ana',
 'fever',
 'leukopenia',
 'thrombocytopenia',
 'auto_immune_hemolysis',
 'delirium',
 'psychosis',
 'seizure',
 'non_scarring_alopecia',
 'oral_ulcers',
 'cutaneous_lupus',
 'pleural_effusion',
 'pericardial_effusion',
 'acute_pericarditis',
 'joint_involvement',
 'proteinuria',
 'anti_cardioliphin_antibodies',
 'anti_β2gp1_antibodies',
 'lupus_anti_coagulant',
 'low_c3',
 'low_c4',
 'anti_dsdna_antibody',
 'anti_smith_antibody']

In [10]:
# for steps in [int(5e4), int(1e5), int(3e5), int(5e5), int(8e5), int(1e6), int(2e6)]:    
# # for steps in [int(1.1e5)]:  
#     dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, f'../models/26_jan/dqn_basic_{steps}')

In [11]:
steps = 11279933
dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, f'../models/24_feb/dqn_basic_{steps}')

using stable baselines 3
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 7.31     |
|    ep_rew_mean      | -0.856   |
|    exploration_rate | 0.539    |
|    success_rate     | 0.18     |
| time/               |          |
|    episodes         | 100000   |
|    fps              | 2717     |
|    time_elapsed     | 201      |
|    total_timesteps  | 547451   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0644   |
|    n_updates        | 124362   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10.9     |
|    ep_rew_mean      | 0.529    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.73     |
| time/               |          |
|    episodes         | 200000   |
|    fps              | 2190     |
|    t

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.24     |
|    ep_rew_mean      | 1.43     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.89     |
| time/               |          |
|    episodes         | 1600000  |
|    fps              | 2066     |
|    time_elapsed     | 3928     |
|    total_timesteps  | 8117104  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00867  |
|    n_updates        | 2016775  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.61     |
|    ep_rew_mean      | 1.35     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.84     |
| time/               |          |
|    episodes         | 1700000  |
|    fps              | 2080     |
|    time_elapsed     | 4102     |
|    total_timesteps  | 8534011  |
| train/              |          |
|    learning_rate  

In [ ]:
#0.92 at 11,279,933 timesteps and 1900000 episodes also at 36,144,785 timestepsc(7800000 episodes)